## Etapa atual
- **Validação da ideia**: criação de MVP manual para comparação de preços.  
- **Criação de base inicial**: coletar dados de algumas notas fiscais ou produtos.  
- **Objetivo desta etapa**: ter **uma base inicial de preços** para análises e comparativos.

##  Estrutura para etapa atual
1. Criar **script para raspar dados das notas fiscais**:  
   - Extrair: produto, preço, mercado, data.  
2. Alimentar **planilha ou banco de dados simples** (CSV / Google Sheets / SQLite).  
3. Fazer **comparativo de preços iniciais** usando esses dados.  
4. Validar se o MVP manual fornece resultados confiáveis antes de automatizar scraping de sites ou QR Codes.


In [131]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [135]:
url = "https://www.nfce.fazenda.sp.gov.br/NFCeConsultaPublica/Paginas/ConsultaQRCode.aspx?p=35250944914620000101651070000018431107018733|2|1|1|6D1F127B092ACFC5A9130DD51D27B2DA25F4B2A7"

response = requests.get(url)

html = response.text

soup = BeautifulSoup(html, 'html.parser')

In [136]:
nome_produto = soup.find_all('span', class_='txtTit')

nomes_produtos = [p.get_text(strip=True) for p in nome_produto]

nomes_produtos


['CALDO SEARA 300GR VERDE CALABRESA',
 'CALDO SEARA 300GR MANDIOQUINHA FRANGO',
 'SEARA NETFLIX 245GR MINI HOT DOG COREANO',
 'CHOCOLATE SNICKERS 42GR DARK',
 'CHOCOLATE SNICKERS 45GR',
 'REFRIG COCA COLA PET 2L']

In [137]:
quantidade = soup.find_all('span', class_='Rqtd')

quantidade_comprada = [p.get_text(strip=True).replace("Qtde.:", "") for p in quantidade]

quantidade_comprada

['1', '1', '1', '1', '1', '1']

In [138]:
unidade = soup.find_all('span', class_='RUN')

unidade_produto = [p.get_text(strip=True).replace("UN:", "")for p in unidade]

In [139]:
valor_unitario = soup.find_all('span',class_='RvlUnit')

vl_unitario = [p.get_text(strip=True).replace("Vl. Unit.:", "") for p in valor_unitario]
vl_unitario

['17,99', '17,99', '18,99', '5,99', '5,99', '10,99']

In [140]:
valor_total = soup.find_all('td',class_='txtTit noWrap')

vl_total = [p.get_text(strip=True).replace("Vl. Total", "") for p in valor_total]

vl_total

['17,99', '17,99', '18,99', '5,99', '5,99', '10,99']

In [141]:
import pandas as pd

dados_dicionario = {
     'Produto': nomes_produtos,
     'Qtde': quantidade_comprada,
     'UN': unidade_produto,
     'Vl. Unit.':vl_unitario,
     'Vl. Total': vl_total

}
df = pd.DataFrame(dados_dicionario)
df

,Produto,Qtde,UN,Vl. Unit.,Vl. Total
0,CALDO SEARA 300GR VERDE CALABRESA,1,UN0001,"17,99","17,99"
1,CALDO SEARA 300GR MANDIOQUINHA FRANGO,1,UN0001,"17,99","17,99"
2,SEARA NETFLIX 245GR MINI HOT DOG COREANO,1,UN0001,"18,99","18,99"
3,CHOCOLATE SNICKERS 42GR DARK,1,UN0001,"5,99","5,99"
4,CHOCOLATE SNICKERS 45GR,1,UN0001,"5,99","5,99"
5,REFRIG COCA COLA PET 2L,1,UN0001,"10,99","10,99"


In [142]:
df['Vl. Total'] = df['Vl. Total'].str.replace(",", ".").astype(float)

In [147]:
total = df['Vl. Total'].mean().round(2)

print(total)

12.99
